### Import Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import sys
import csv

import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import io
import warnings
warnings.filterwarnings('ignore')


### Urls to scrape the data

In [2]:
#urls
main_url='https://www.prokabaddi.com'
stats_all = 'https://www.prokabaddi.com/stats'
url_alt = 'https://www.sportskeeda.com/go/pro-kabaddi/stats'

headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}

In [3]:
#Get the details from the main page of Pro kabaddi.

response = requests.get(main_url, headers=headers)
print(response)

<Response [200]>


### Extract links for all the team from main page of pro kabaddi and store into a csv file: 'team_links.csv' on local machine


In [5]:
soup = bs(response.text, 'html.parser')
df_team_links = pd.DataFrame()
links = soup.find_all('a')

team_links = [main_url + x.attrs['href'] for x in links[13:25]]
team_name = [x.text for x in links[13:25]]
df_team_links['team_name'] = team_name
df_team_links['team_url_link'] = team_links
df_team_links.to_csv('team_links.csv', index=False)

In [48]:
!pip install selenium

** https://medium.com/shakuro/adopting-ipython-jupyter-for-selenium-testing-d02309dd00b8

In [4]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
d = webdriver.Chrome(executable_path=r'C:\webdrivers\chromedriver.exe',chrome_options=chrome_options)

### Extract team stats for each team and for every seasons and store into CSV file

In [7]:
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import ElementNotInteractableException

In [33]:
ext_data = []
i = 0
print(team_links)
for m in team_links:
    time.sleep(5)
    d.maximize_window()
    html = d.page_source

    soup_1 = bs(html, 'html.parser')
    cols  = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-tbl-header'}).find_all('div', {'class': 'si-data-block'})
    main_cols = [x.text for x in cols]
    cols1 = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-overall'}).find_all('div', {'class': 'si-data-block'})
    cols_x = [x.text for x in cols1]
    cols2 = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-attack'}).find_all('div', {'class': 'si-data-block'})
    cols2_l = [x.text for x in cols2]
    cols3 = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-defence'}).find_all('div', {'class': 'si-data-block'})
    cols3_l = [x.text for x in cols3]
    main_cols = main_cols + cols_x + cols2_l + cols3_l
    team_cols = ['team_name']
    main_cols = team_cols + main_cols

    datahead = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-right'}).find('div', {'class': 'si-tbl-header'}).find_all('div', {'class': 'si-data-block'})
    datahead_t = [x.text for x in datahead]

    data = []
    for each1 in range(1,4):
        list1 = []
        for each in range(len(datahead_t)):
            datahead1 = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-right'}).find_all('div', {'class': 'si-tbl-wraper'})[each1].find_all('div', {'class': 'si-tbl-data'})[each].find_all('div', {'class': 'si-data-block'})
            datahead1_t = [x.text.strip('\n').split('\n')[0] for x in datahead1]
            list1.append(datahead1_t)
        data.append(list1)

    newdata=[]
    for each in range(len(data[0])):
        newdata.append(data[0][each] + data[1][each] + data[2][each])

    newdata1=[]
    for each in range(len(newdata)):
        temp = []
        temp.append(team_name[i])
        temp.append(datahead_t[each])
        for inside in range(len(newdata[0])):
            temp.append(newdata[each][inside])
        newdata1.append(temp)

    ext_data.append(newdata1)
    i = i + 1


['https://www.prokabaddi.com/teams/bengal-warriors-profile-4', 'https://www.prokabaddi.com/teams/bengaluru-bulls-profile-1', 'https://www.prokabaddi.com/teams/dabang-delhi-kc-profile-2', 'https://www.prokabaddi.com/teams/gujarat-fortunegiants-profile-31', 'https://www.prokabaddi.com/teams/haryana-steelers-profile-28', 'https://www.prokabaddi.com/teams/jaipur-pink-panthers-profile-3', 'https://www.prokabaddi.com/teams/patna-pirates-profile-6', 'https://www.prokabaddi.com/teams/puneri-paltan-profile-7', 'https://www.prokabaddi.com/teams/tamil-thalaivas-profile-29', 'https://www.prokabaddi.com/teams/telugu-titans-profile-8', 'https://www.prokabaddi.com/teams/u-mumba-profile-5', 'https://www.prokabaddi.com/teams/up-yoddha-profile-30']
https://www.prokabaddi.com/teams/bengal-warriors-profile-4
https://www.prokabaddi.com/teams/bengaluru-bulls-profile-1
https://www.prokabaddi.com/teams/dabang-delhi-kc-profile-2
https://www.prokabaddi.com/teams/gujarat-fortunegiants-profile-31
https://www.prok

In [34]:
ext_data1 = []
for each in ext_data:
    ext_data1 = ext_data1 + each

df_matches_det = pd.DataFrame(data=ext_data1, columns=main_cols)
df_matches_det.head()
df_matches_det.to_csv('team_stats.csv', index=False)

'\nwith open(\'team_stats.csv\', \'w\') as f:\n  employee_writer = csv.writer(f, delimiter=\',\', quotechar=\'"\', quoting=csv.QUOTE_MINIMAL)\n  employee_writer.writerow([\'TEAM NAME\', \'SEASONS\',\'MATCHES PLAYED\',\'WINS\',\'DRAWS\',\'LOSSES\',\'FINISHING POSITION\',\'HIGHEST SCORE\',\'BIGGEST WINNING MARGIN\',\n                              \'TOTAL RAIDS\',\'SUCCESSFUL RAIDS\',\'UNSUCCESSFUL RAID\',\'EMPTY RAIDS\',\'SUCCESS RAID %\',\'NO. OF SUPER RAIDS\',\'RAID TOUCH POINTS\',\n                              \'RAID BONUS POINTS\',\'TOTAL RAID POINTS\'\n                              \'TOTAL TACKLES\',\'SUCCESSFUL TACKLES\',\'UNSUCCESFUL TACKLES\',\'SUCCESSFUL TACKLE %\',\'NO. OF SUPER TACKLES\',\'ALL OUTS INFLICTED\',\n                              \'TOTAL ALL OUT POINTS\',\'TOTAL DEFENCE POINTS\'])\n\n  for each in range(len(df_matches_det)):\n    row1 = df_matches_det.iloc[each].values.flatten().tolist()\n    #print(row1)\n    employee_writer.writerow(row1)\n    \nfiles.download(\

### Get All schedules and results of every match of Pro Kabaddi for all seasons

In [9]:
divhead = soup.find('div', {'class': 'nav-middle'})
divli = divhead.find('li', {'class': 'schedule-fixtures-results'}).find('a').attrs['href']
schedule_link = main_url + divli


In [39]:
d.get(schedule_link)
time.sleep(5)
d.maximize_window()
html = d.page_source

select = Select(d.find_element_by_class_name('si-season-change'))

data=[]
cols = ['Season', 'Date', 'Time', 'Status', 'Team1', 'Score1', 'Team2', 'Score2', 'Venue', 'Match-Results']

for each_indx in range(1,7):
    select.select_by_index(each_indx)
    time.sleep(5)
    soup_1 = bs(d.page_source)
    div_1= soup_1.find_all('div', {'class':'si-fix-box'})
  
    for each in div_1:
        list1=[]
        if each_indx==1:
            list1.append('Season 6')
        elif each_indx==2:
            list1.append('Season 5')
        elif each_indx==2:
            list1.append('Season 4')
        elif each_indx==2:
            list1.append('Season 3')
        elif each_indx==2:
            list1.append('Season 2')
        else:
            list1.append('Season 1')
    
        list1.append(each.find('div', {'class':'matDate'}).text.strip('\n'))
        list1.append(each.find('div', {'class':'matTime'}).text.strip('\n'))
        list1.append(each.find('div', {'class':'matStatus'}).text.strip('\n'))
        list1.append("".join(each.find_all('div', {'class':'fix-teamName'})[0].find('div', {'class':'si-fullName'}).text.split('\n')))
        list1.append(each.find('div', {'class':'matScore'}).find_all('span', {'class': 'si-score'})[0].text.strip())
        list1.append("".join(each.find_all('div', {'class':'fix-teamName'})[1].find('div', {'class':'si-fullName'}).text.split('\n')))
        list1.append(each.find('div', {'class':'matScore'}).find_all('span', {'class': 'si-score'})[1].text.strip())
        list1.append(each.find('div', {'class':'mat-venue'}).text.strip('\n'))
        list1.append(each.find('div', {'class':'mat-resultStatus'}).text.strip('\n'))
        data.append(list1)

match_res = pd.DataFrame(data=data, columns=cols)
match_res.shape

(515, 10)

In [40]:
match_res.to_csv('match_res.csv', index=False)

In [10]:
d.get(schedule_link)
time.sleep(5)
d.maximize_window()
html = d.page_source

select = Select(d.find_element_by_class_name('si-season-change'))
select.select_by_index(0)
time.sleep(5)

select_1 = Select(d.find_element_by_class_name('si-months-change'))
data=[]
cols = ['Season', 'Date', 'Time', 'Status', 'Team1', 'Score1', 'Team2', 'Score2', 'Venue', 'Match-Results']

for each_indx in range(1,5):
  
    select_1.select_by_index(each_indx)
    time.sleep(5)
    soup_1 = bs(d.page_source)
    div_1= soup_1.find_all('div', {'class':'si-fix-box'})
  
    for each in div_1:
        list1 = []
        list1.append('Season 7')
    
        list1.append(each.find('div', {'class':'matDate'}).text.strip('\n'))
        list1.append(each.find('div', {'class':'matTime'}).text.strip('\n'))
        list1.append(each.find('div', {'class':'matStatus'}).text.strip('\n'))
        list1.append("".join(each.find_all('div', {'class':'fix-teamName'})[0].find('div', {'class':'si-fullName'}).text.split('\n')))
        list1.append(each.find('div', {'class':'matScore'}).find_all('span', {'class': 'si-score'})[0].text.strip())
        list1.append("".join(each.find_all('div', {'class':'fix-teamName'})[1].find('div', {'class':'si-fullName'}).text.split('\n')))
        list1.append(each.find('div', {'class':'matScore'}).find_all('span', {'class': 'si-score'})[1].text.strip())
        list1.append(each.find('div', {'class':'mat-venue'}).text.strip('\n'))
        list1.append(each.find('div', {'class':'mat-resultStatus'}).text.strip('\n'))
        data.append(list1)

match_sch = pd.DataFrame(data=data, columns=cols)
match_sch.shape

(137, 10)

In [11]:
match_sch.to_csv('match_sch.csv', index=False)

### Extract Player stats from Pro Kabaddi

In [51]:
d.get(stats_all)
time.sleep(5)
d.maximize_window()
html = d.page_source


soup = bs(html, 'html.parser')
#Player stats links with respect to its matrics
playerstat_div = soup.find_all('div', {'class': 'si-leadBoard-box'})
metrics = [each.text.split('\n')[3] for each in playerstat_div]

player_stats_urls = []
for each in playerstat_div:
    player_stat = each.find('div', {'class': 'si-leadBoard-viewMore'})
    player_stats_urls.append(main_url + player_stat.find('a').attrs['href'])

player_stats_det = pd.DataFrame()
player_stats_det['player_metrics'] = metrics
player_stats_det['player_stats_url'] = player_stats_urls
player_stats_det



,player_metrics,player_stats_url
0,Total Points,https://www.prokabaddi.com/stats/49-102-total-...
1,Raid Points,https://www.prokabaddi.com/stats/49-22-raid-po...
2,Tackle Points,https://www.prokabaddi.com/stats/49-103-tackle...
3,Successful Raids,https://www.prokabaddi.com/stats/49-21-success...
4,Super Raids,https://www.prokabaddi.com/stats/49-104-super-...
5,Super 10s,https://www.prokabaddi.com/stats/49-100-super-...
6,Successful Tackles,https://www.prokabaddi.com/stats/49-23-success...
7,Super Tackles,https://www.prokabaddi.com/stats/49-28-super-t...
8,High 5s,https://www.prokabaddi.com/stats/49-101-high-5...


In [42]:
player_stats_det.to_csv('player_stats_urls.csv', index=False)

In [52]:
main_data = []
p = 0
for each_url in player_stats_det['player_stats_url']:
    print(each_url)
    d.get(each_url)
    time.sleep(5)
    d.maximize_window()
    element = d.find_element_by_id('player_Btn')
    element.click()
    time.sleep(5)

    select = Select(d.find_element_by_id('season_dropdown'))
  
    options_det = [o.text for o in select.options]
    data1 = []
    for each_index in range(1, len(options_det)):
        print(options_det[each_index])
        select.select_by_index(each_index)

        time.sleep(5)

        load_more = d.find_element_by_id('load_more')
    
        while load_more:
            try:
                load_more.click()
                time.sleep(5)
                d.maximize_window()
                load_more = d.find_element_by_id('load_more')
            except ElementNotInteractableException:
                break

        soup_1 = bs(d.page_source)
    
        #player_cols_div = soup_1.find('div',attrs={'class':'si-leadBoard-detail-tRow'})
        #player_cols_div_det = player_cols_div.find_all('div',attrs={'class':'si-leadBoard-detail-tCol'})
        #print(len(player_cols_div_det))
        #newcols = [each.text.split('\n')[2] for each in player_cols_div_det]
        #print(newcols)
    
        player_div = soup_1.findAll('div',attrs={'class':'sipk-lb-detailItem'})

    
        print(len(player_div))
        data = []
        for each in player_div:
            list1 = []
      
            list1.append(options_det[each_index])
            list1.append(player_stats_det['player_metrics'][p])
      
            pdet = each.find('div', {'class':'sipk-lb-playerName'})
            if pdet:
                list1.append(pdet.text.split('\n')[3].strip())
            else:
                list1.append(np.NAN)

            pdet = each.find('a', {'class':'sipk-lb-team'})
            if pdet:
                list1.append(" ".join(pdet.attrs['href'].split('/')[2].split('-')[0:2]))
            else:
                list1.append(np.NAN)

            pdet = each.find('div', {'class':'sipk-lb-matchedPlayed'})
            if pdet:
                list1.append(pdet.text.split('\n')[1])
            else:
                list1.append(np.NAN)

            pdet = each.find('div', {'class':'sipk-lb-raidPoints'})
            if pdet:
                list1.append(pdet.text.split('\n')[1])
            else:
                list1.append(np.NAN)

            pdet = each.find('div', {'class':'sipk-lb-playerprofile'})
            if pdet:
                list1.append(pdet.text.split('\n')[1].strip())
            else:
                list1.append(np.NAN)

            data.append(list1)

        data1.append(data)
    main_data.append(data1)
    p = p + 1
  


https://www.prokabaddi.com/stats/49-102-total-points-statistics
SEASON 7
171
SEASON 6
186
SEASON 5
181
SEASON 4
104
SEASON 3
122
SEASON 2
128
SEASON 1
96
https://www.prokabaddi.com/stats/49-22-raid-points-statistics
SEASON 7
112
SEASON 6
125
SEASON 5
122
SEASON 4
71
SEASON 3
84
SEASON 2
88
SEASON 1
71
https://www.prokabaddi.com/stats/49-103-tackle-points-statistics
SEASON 7
144
SEASON 6
154
SEASON 5
150
SEASON 4
80
SEASON 3
99
SEASON 2
107
SEASON 1
85
https://www.prokabaddi.com/stats/49-21-successful-raids-statistics
SEASON 7
105
SEASON 6
119
SEASON 5
112
SEASON 4
67
SEASON 3
78
SEASON 2
83
SEASON 1
70
https://www.prokabaddi.com/stats/49-104-super-raids-statistics
SEASON 7
40
SEASON 6
57
SEASON 5
54
SEASON 4
28
SEASON 3
28
SEASON 2
33
SEASON 1
29
https://www.prokabaddi.com/stats/49-100-super-10s-statistics
SEASON 7
34
SEASON 6
31
SEASON 5
33
SEASON 4
15
SEASON 3
17
SEASON 2
16
SEASON 1
20
https://www.prokabaddi.com/stats/49-23-successful-tackles-statistics
SEASON 7
144
SEASON 6
154
SEA

In [53]:
main_data_final = []
for x in main_data:
    for y in x:
        for z in y:
            main_data_final.append(z)
    
cols = ['Season_Name', 'Metrics', 'Player_Name', 'Team_Name', 'Total_matches_played', 'Points', 'Player_type']
player_stat = pd.DataFrame(data=main_data_final, columns=cols)

player_stat.to_csv('player_stat.csv', index=False)


### Standings for each team in current season as of now

In [5]:
soup = bs(response.text, 'html.parser')
divhead = soup.find('div', {'class': 'nav-middle'})
divli = divhead.find('li', {'class': 'standings'}).find('a').attrs['href']
standings_link = main_url + divli
print(standings_link)

https://www.prokabaddi.com/standings


In [9]:
d.get(standings_link)
time.sleep(5)
d.maximize_window()
html = d.page_source

select = Select(d.find_element_by_id('season_selectBox'))
time.sleep(5)

data=[]
cols = ['Season', 'Rank', 'Team_Name', 'Matches_Played', 'Matches_Won', 'Matches_Lost', 'Matches_Drow', 'Total_Points']

select.select_by_index(0)
time.sleep(5)
d.maximize_window()
soup_1 = bs(d.page_source)
div_1= soup_1.find_all('div', {'class':'sipk-standing-dataRow'})
  
for each in div_1:
    list1 = []
    list1.append('Season 7')
      
    list1.append(each.find('div', {'class':'sipk-rank'}).text.strip('\n').split('\n')[0])
    list1.append(each.find('div', {'class':'sipk-team'}).text.strip('\n'))
    list1.append(each.find('div', {'class':'sipk-play'}).text.strip('\n'))
    list1.append(each.find('div', {'class':'sipk-won'}).text.strip('\n'))
    list1.append(each.find('div', {'class':'sipk-lost'}).text.strip('\n'))
    list1.append(each.find('div', {'class':'sipk-drow'}).text.strip('\n'))
    list1.append(each.find('div', {'class':'sipk-points'}).text.strip('\n'))

    data.append(list1)

team_rank = pd.DataFrame(data=data, columns=cols)
team_rank.to_csv('team_rank.csv', index=False)